In [2]:
import pandas as pd
import numpy as np
import mysql.connector as sqlconnector

In [91]:
connection = connection = sqlconnector.connect(host='192.168.1.73',
                           database='presensi_epsindo',
                           user='root',
                           password='epsindo123')

In [65]:
def load_department(conn) :
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM department")
    result = cursor.fetchall()
    
    name = [result[i][1] for i in range(len(result))]
    ids = [result[i][0] for i in range(len(result))]

    department_df = pd.DataFrame({'id':ids, 'name':name})
    return department_df

load_department(connection)

,id,name
0,0,Warehouse
1,1,Other
2,2,Finance & Accounting
3,3,Sales Department
4,4,Service & Support Department
5,5,Marketing
6,6,Product Manager


In [20]:
df_participant
cursor = connection.cursor()

sql = "INSERT INTO participant (participant_id, name, department_id, email)  VALUES (%s, %s, %s, %s)"
values = []
for ind,val in df_participant.iterrows() :
    values.append((val['id_p'], val['Name'], val['dep_id'], val['email']))
print(values)
cursor.executemany(sql, values)
connection.commit()
print(cursor.rowcount, "rows executed.")

[(0, 'Chris Ariwibowo', 0, 'None'), (1, 'Mirsan Mirsan', 0, 'None'), (2, 'Slamat Riadi', 0, 'None'), (3, 'Sumarno Sumarno', 0, 'None'), (4, 'Bayu Bayu', 1, 'None'), (5, 'Yuni Aryani', 2, 'None'), (6, 'Jumyati Jumyati', 3, 'None'), (7, 'Ineke Sri Lestari', 2, 'None'), (8, 'Cahyanto Cahyanto', 4, 'None'), (9, 'Fachrudin Ali', 5, 'None'), (10, 'Nuryanti Nuryanti', 2, 'None'), (11, 'Setia Wayah Gutama', 4, 'None'), (12, 'harun boerhan', 4, 'None'), (13, 'Nurahdiatna Subagya', 4, 'None'), (14, 'Mark Kenny Williams', 3, 'None'), (15, 'Muh. Agung Fasrulhaq', 4, 'None'), (16, 'raisul amin', 4, 'None'), (17, 'Melissa Christina Tuwoh', 3, 'None'), (18, 'Lina Wirahadi', 6, 'None'), (19, 'Dwi Septia Ningsih', 0, 'None'), (20, 'Roby Roby', 1, 'None'), (21, 'Arrent Ong', 6, 'None')]


TypeError: 'int' object is not callable

In [21]:
print(cursor.rowcount, "rows executed.")

22 rows executed.


In [28]:
def load_participants(conn, department=-1):
    cursor = conn.cursor()

    if department < 0 :
        cursor.execute("SELECT * FROM participant")
    else :
        cursor.execute("SELECT * FROM participant WHERE department_id="+str(department))
    
    result = cursor.fetchall()
    
    name = [result[i][1] for i in range(len(result))]
    ids = [result[i][0] for i in range(len(result))]
    dep_ids = [result[i][2] for i in range(len(result))]
    emails = [result[i][3] for i in range(len(result))]

    participant_df = pd.DataFrame({'id':ids, 'name':name, 'dep_id':dep_ids, 'email':emails})
    return participant_df

load_participants(connection, department=5)

,id,name,dep_id,email
0,9,Fachrudin Ali,5,None


In [62]:
df_presensi = df_presensi.drop(columns=['ID'])
# df_presensi = df_presensi.fillna()
df_presensi['First Check In'] = pd.to_datetime(df['First Check In'], format='%I:%M %p').dt.strftime('%H:%M:%S')
df_presensi['Last Check Out'] = pd.to_datetime(df['Last Check Out'], format='%I:%M %p').dt.strftime('%H:%M:%S')
df_presensi = df_presensi.fillna(np.nan).replace([np.nan], [None])
df_presensi

,Date,Participant_id,Status,First Check In,Last Check Out
7,2022-07-26,0,IN,09:53:00,None
8,2022-07-26,1,IN,09:50:00,None
9,2022-07-26,2,IN,09:22:00,None
10,2022-07-26,3,IN,09:22:00,None
11,2022-07-26,4,IN,08:41:00,None
...,...,...,...,...,...
323,2022-07-01,11,Present,10:05:00,17:03:00
324,2022-07-01,18,Present,10:01:00,17:02:00
325,2022-07-01,13,Present,09:56:00,17:02:00
326,2022-07-01,15,Present,09:59:00,17:05:00


In [68]:
cursor = connection.cursor()

sql = "INSERT INTO presensi_table (date, participant_id, status, firstcheckin, lastcheckout)  VALUES (%s, %s, %s, %s, %s)"
values = []
for ind,val in df_presensi.iterrows() :
    # print(val.values)
    values = tuple(val.values)
    cursor.execute(sql, values)
connection.commit()
print(cursor.rowcount, "rows executed.")

1 rows executed.


In [89]:
from math import floor
from datetime import date


def format_timedelta(td):
    if pd.isnull(td) :
        return None
    else :
        minutes, seconds = divmod(td.seconds + td.days * 86400, 60)
        hours, minutes = divmod(minutes, 60)
        return '{:d}:{:02d}:{:02d}'.format(hours, minutes, seconds)

def total_late(fi) :
    if fi.total_seconds()/3600 >= 10:
        tolate = (fi.total_seconds() - (3600*10))/60
        return str(tolate) + " Minute"
    else :
        return 0

def load_presensi(conn, month='January', year=date.today().year):
    cursor = conn.cursor()
    query = "SELECT * FROM presensi_table"
    cursor.execute(query)

    result = cursor.fetchall()
    df_presensi = pd.DataFrame(data=result, columns=['id','Date', 'Participant_id','status','firstcheckin', 'lastcheckout'])
    df_presensi['firstcheckinstr'] = df_presensi['firstcheckin'].apply(lambda x : format_timedelta(x))
    df_presensi['lastcheckoutstr'] = df_presensi['lastcheckout'].apply(lambda x : format_timedelta(x))
    df_presensi['total_late'] = df_presensi['firstcheckin'].apply(lambda x: total_late(x))
    return df_presensi

load_presensi(connection)

,id,Date,Participant_id,status,firstcheckin,lastcheckout,firstcheckinstr,lastcheckoutstr,total_late
0,1,2022-07-26,0,IN,0 days 09:53:00,NaT,9:53:00,None,0
1,2,2022-07-26,1,IN,0 days 09:50:00,NaT,9:50:00,None,0
2,3,2022-07-26,2,IN,0 days 09:22:00,NaT,9:22:00,None,0
3,4,2022-07-26,3,IN,0 days 09:22:00,NaT,9:22:00,None,0
4,5,2022-07-26,4,IN,0 days 08:41:00,NaT,8:41:00,None,0
...,...,...,...,...,...,...,...,...,...
316,317,2022-07-01,11,Present,0 days 10:05:00,0 days 17:03:00,10:05:00,17:03:00,5.0 Minute
317,318,2022-07-01,18,Present,0 days 10:01:00,0 days 17:02:00,10:01:00,17:02:00,1.0 Minute
318,319,2022-07-01,13,Present,0 days 09:56:00,0 days 17:02:00,9:56:00,17:02:00,0
319,320,2022-07-01,15,Present,0 days 09:59:00,0 days 17:05:00,9:59:00,17:05:00,0


In [90]:
df_leave

,ID,Name,Applied At,Leave,From,To,For,Session,Status,Emergency,Reason,Remark,Department,Branch,Attachment
0,1007,Melissa Christina Tuwoh,2022-07-14,ANNUAL,2022-07-29,2022-07-29,1.00 Days,Full Day,Pending Approval,NaN,Acara keluarga,NaN,Sales Department,NaN,NaN
1,4,Dwi Septia Ningsih,2022-07-19,ANNUAL,2022-07-25,2022-07-26,2.00 Days,Full Day,Approved,NaN,Ada urusan keluarga,NaN,Warehouse,NaN,NaN
2,1004,Cahyanto Cahyanto,2022-07-15,ANNUAL,2022-07-15,2022-07-15,1.00 Days,Full Day,Approved,Yes,Antar ibu ke poli Mata Katarak,NaN,Service & Support Department,NaN,√
3,1006,Nuryanti Nuryanti,2022-07-19,ANNUAL,2022-07-15,2022-07-15,1.00 Days,Full Day,Pending Approval,Yes,Sakit,NaN,Finance & Accounting,NaN,NaN
4,1002,Jumyati Jumyati,2022-07-14,ANNUAL,2022-07-15,2022-07-18,2.00 Days,Full Day,Approved,NaN,Ada keperluan keluarga,NaN,Sales Department,NaN,NaN
5,1,Chris Ariwibowo,2022-07-08,Bereavement,2022-07-11,2022-07-11,1.00 Days,Full Day,Approved,NaN,Keluarga,NaN,Warehouse,NaN,NaN
6,1013,Nurahdiatna Subagya,2022-06-25,ANNUAL,2022-07-08,2022-07-08,1.00 Days,Full Day,Cancelled,NaN,Pulang kampung idul adha,NaN,Service & Support Department,NaN,NaN
7,3,Slamat Riadi,2022-07-05,ANNUAL,2022-07-07,2022-07-07,1.00 Days,Full Day,Approved,NaN,Ada urusan keluarga,NaN,Warehouse,NaN,NaN
8,1014,Mark Kenny Williams,2022-06-23,ANNUAL,2022-06-27,2022-07-05,7.00 Days,Full Day,Approved,NaN,Family visit Canada,NaN,Sales Department,NaN,NaN


In [94]:
    df_leave_tb = df_leave_tb.drop(columns=['id_leave'])
    df_leave_tb

,participant_id,leave_kind,from,to,reason
0,17,ANNUAL,2022-07-29,2022-07-29,Acara keluarga
1,19,ANNUAL,2022-07-25,2022-07-26,Ada urusan keluarga
2,8,ANNUAL,2022-07-15,2022-07-15,Antar ibu ke poli Mata Katarak
3,10,ANNUAL,2022-07-15,2022-07-15,Sakit
4,6,ANNUAL,2022-07-15,2022-07-18,Ada keperluan keluarga
5,0,Bereavement,2022-07-11,2022-07-11,Keluarga
6,13,ANNUAL,2022-07-08,2022-07-08,Pulang kampung idul adha
7,2,ANNUAL,2022-07-07,2022-07-07,Ada urusan keluarga
8,14,ANNUAL,2022-06-27,2022-07-05,Family visit Canada


In [96]:
cursor = connection.cursor()

leave_values = []
for ind, val in df_leave_tb.iterrows():
    leave_values.append(tuple(val.values))
sql = "INSERT INTO leave_table (participant_id, leave_kind, leave_from, leave_to, reason) VALUES(%s, %s, %s, %s, %s)"
cursor.executemany(sql, leave_values)
connection.commit()

print(cursor.rowcount, " rows inserted.")


9  rows inserted.
